In [57]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

In [58]:
calico_url = "https://nsdc25.calicotab.com/NordicSDC2025"
motions_url = f"{calico_url}/motions/statistics"

In [59]:
response = requests.get(motions_url) 

html = response.text
soup = BeautifulSoup(html, 'html.parser')
soup.prettify()

'<!DOCTYPE html>\n<html lang="en">\n <head>\n  <meta charset="utf-8"/>\n  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>\n  <meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>\n  <title>\n   NordicSDC2025 | Motion Statistics\n  </title>\n  <meta content="The tab for NordicSDC2025 runs on Calico - Tabbycat, a source-available tab system for a variety of parliamentary debating formats" name="description">\n   <meta content="The tab for NordicSDC2025 runs on Calico - Tabbycat, a source-available tab system for a variety of parliamentary debating formats" property="og:description">\n    <meta content="https://calico-static.s3.us-east-2.amazonaws.com/logo-social.png" property="og:image">\n     <link as="font" href="https://calico-static.s3.us-east-2.amazonaws.com/fonts/Inter-Regular.woff2" rel="preload" type="font/woff2"/>\n     <link href="https://calico-static.s3.us-east-2.amazonaws.com/logo-32x32.png" rel="icon" sizes="32x32" type="image/png">\

In [60]:
motion_divs = soup.find_all('div', class_="list-group mt-3")

print(f"Found {len(motion_divs)} motions.")

Found 11 motions.


In [ ]:
data = {
    "Motion": [],
    "Round": [],
    "Info Slide": [],

    "Prop wins": [],
    "Opp wins": [],
    "Balance": [],
}

def parse_motion(div): 
    round = div.find('span', class_="badge").get_text(strip=True)
    motion = div.find('h4').get_text(strip=True)

    info_slide = div.find(string=re.compile("View Info Slide"))
    info_slide_text = None

    if info_slide is not None:
        info_slide_text = info_slide.find_next('div', class_='modal-body').get_text(strip=True)

    # regex match the wins for prop and opp
    text = div.get_text()

    re_prop = re.compile(r"(\d+) Prop win")
    re_opp = re.compile(r"(\d+) Opp win")

    prop_wins = re.findall(re_prop, text)[0]
    opp_wins = re.findall(re_opp, text)[0]

    balance = div.find(string=re.compile('balance')).get_text(strip=True)

    return motion, round, info_slide_text, prop_wins, opp_wins, balance

for div in motion_divs:
    motion, round, info_slide_text, prop_wins, opp_wins, balance = parse_motion(div)

    data["Motion"].append(motion)
    data["Round"].append(round)
    data["Info Slide"].append(info_slide_text)
    data["Prop wins"].append(prop_wins)
    data["Opp wins"].append(opp_wins)
    data["Balance"].append(balance)

In [67]:
df = pd.DataFrame(data)
df.head()

,Motion,Round,Info Slide,Prop wins,Opp wins,Balance
0,This House Supports gentle parenting becoming ...,Round 1,Gentle parenting is a parenting style that say...,16,11,imbalanced at 50% level
1,THR the creation of the series and film indust...,Round 2,None,13,14,probably balanced
2,This House Believes That it is in the interest...,Round 3,Democratic backsliding is a process of regime ...,15,12,probably balanced
3,This House Opposes the Globalization of the Ma...,Round 4,“Major European Football Leagues” include the ...,10,16,imbalanced at 50% level
4,This House Would implement a weekly 'blackout ...,Round 5,A 'blackout day' refers to a day where all soc...,13,13,probably balanced
